# Chapter 8: Avoiding Hallucinations

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
import re
from mlx_lm import load, stream_generate

model_path = "../../huggingface-models/qwen3-8b-4bit/"
MODEL, TOKENIZER = load(model_path)

In [2]:
from IPython.display import Markdown, display

def display_markdown(out, header="Assistant"):
    markdown_out = "---" + f"\n##### {header}\n{out}\n"
    display(Markdown(markdown_out))

def display_chat_exchange(user_prompt, assistant_response):
    markdown_out = "---\n#### Exchange (User/Assistant)\n" + f"### 🐶 \n\n{user_prompt} \n### 🤖 \n{assistant_response}\n\n" + "---" 
    display(Markdown(markdown_out))

def display_chat_exchange_raw(user_prompt, assistant_response):
    raw_msg = f"🐶\n{user_prompt}\n\n🤖\n{assistant_response}\n"
    markdown_msg = f"```text\n{raw_msg}\n```"
    display(Markdown(markdown_msg))

In [ ]:
def get_completion(
        prompt: str, 
        system_prompt="", prefill="", max_tokens=32768, enable_thinking=False, print_stream=False):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": prefill},
    ]
    tokenized_prompt = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        enable_thinking=enable_thinking
    )

    total_response = []

    for response in stream_generate(
        MODEL, 
        TOKENIZER, 
        tokenized_prompt, 
        max_tokens=max_tokens 
    ):
        total_response.append(response.text)
        if print_stream:
            print(response.text, end="")

    return TOKENIZER.decode(tokenized_prompt), "".join(total_response) 

---

## Lesson

Some bad news: **Claude sometimes "hallucinates" and makes claims that are untrue or unjustified**. The good news: there are techniques you can use to minimize hallucinations.
				
Below, we'll go over a few of these techniques, namely:
- Giving Claude the option to say it doesn't know the answer to a question
- Asking Claude to find evidence before answering

However, **there are many methods to avoid hallucinations**, including many of the techniques you've already learned in this course. If Claude hallucinates, experiment with multiple techniques to get Claude to increase its accuracy.

### Examples

Here is a question about general factual knowledge in answer to which **Claude hallucinates several large hippos because it's trying to be as helpful as possible**.

In [ ]:
SYSTEM_PROMPT= ""
PROMPT = "Who is the heaviest hippo of all time?"

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Who is the heaviest hippo of all time?<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Who is the heaviest hippo of all time?

🤖
The heaviest hippo on record is **Benny**, a male hippopotamus who lived at the Cincinnati Zoo in the United States. Benny was born in 1971 and weighed an astonishing **1,628 kilograms (3,590 pounds)** at his heaviest. He was known for his massive size and was a popular attraction at the zoo.

Benny passed away in 2011 at the age of 40, making him one of the oldest and largest hippos ever recorded. His size and longevity made him a notable figure in the world of zoo animals.

While there may be other hippos that have reached large sizes, Benny holds the record for the heaviest hippo ever documented.

```

### Note:
For the question `Who is the heaviest hippo of all time?` Gwen answered the following:

> The heaviest hippo on record is **Benny**, a male hippopotamus who lived at the Cincinnati Zoo in the United States. Benny was born in 1971 and weighed an astonishing **1,628 kilograms (3,590 pounds)** at his heaviest. He was known for his massive size and was a popular attraction at the zoo.

> Benny passed away in 2011 at the age of 40, making him one of the oldest and largest hippos ever recorded. His size and longevity made him a notable figure in the world of zoo animals.

While there may be other hippos that have reached large sizes, Benny holds the record for the heaviest hippo ever documented.

It clearly invented an answer. Though there's a Hippo named Benny, there's no evidence within Google search that indicates that Benny is indeed the heaviest Hippo!

A solution we can try here is to "**give Claude an out**" — tell Claude that it's OK for it to decline to answer, or to only answer if it actually knows the answer with certainty.

In [5]:
SYSTEM_PROMPT= ""
PROMPT = "Who is the heaviest hippo of all time? Only answer if you know the answer with certainty."

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Who is the heaviest hippo of all time? Only answer if you know the answer with certainty.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Who is the heaviest hippo of all time? Only answer if you know the answer with certainty.

🤖
The heaviest hippo on record was a male hippo named "Hector," who lived at the Cincinnati Zoo. He weighed approximately **5,200 pounds (2,358 kilograms)** and was the heaviest hippo ever documented.

```